In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

### Data loading

We will only use the MovieLens 1M dataset for this project, as we will use the movies genres as features.
The IMDb dataset does not provide the ratings of movies, and genres are already present in the MovieLens dataset.

Moreover, the IMDb dataset has much more movies than the MovieLens dataset, and we wouldn't be able to do anything with them.

We could use the IMDb dataset to get cast information, but this would increase the dimension of the features way to much for a recommendation system like this to be efficient.

In [2]:
df_movies = pd.read_csv("ml-1m/movies.dat", sep="::", engine="python", encoding='latin-1', names=['movieId', 'title', 'genres'])
df_ratings = pd.read_csv("ml-1m/ratings.dat", sep="::", engine="python", encoding='latin-1', names=['userId', 'movieId', 'rating', 'timestamp'])
df_users = pd.read_csv("ml-1m/users.dat", sep="::", engine="python", encoding='latin-1', names=['userId', 'gender', 'age', 'occupation', 'zip'])

Clean the movies dataframe

In [3]:
# Remove the year from the title
def remove_year(title):
    if title.split(" ")[-1].startswith("("):
        title = (" ".join(title.split(" ")[:-1])).strip()
    return title

def reorder_title(title):
    if title.split(',')[-1].strip() in ['The', 'A', 'An']:
        title = (title.split(',')[-1].strip() + " " + " ".join(title.split(',')[:-1])).strip()
    return title

def lower_case(title):
    return title.lower()

def clean_title(title):
    title = remove_year(title)
    title = reorder_title(title)
    title = lower_case(title)
    return title

df_movies['title'] = df_movies['title'].apply(clean_title)

df_movies

,movieId,title,genres
0,1,toy story,Animation|Children's|Comedy
1,2,jumanji,Adventure|Children's|Fantasy
2,3,grumpier old men,Comedy|Romance
3,4,waiting to exhale,Comedy|Drama
4,5,father of the bride part ii,Comedy
...,...,...,...
3878,3948,meet the parents,Comedy
3879,3949,requiem for a dream,Drama
3880,3950,tigerland,Drama
3881,3951,two family house,Drama


In [4]:
# Explode the 'genres' column

df_movies['genres'] = df_movies['genres'].str.split('|')

df_movies_exploded = df_movies.explode('genres')

df_movies_one_hot = pd.get_dummies(df_movies_exploded['genres'])

df_movies_one_hot_grouped = df_movies_one_hot.groupby(df_movies_exploded.index).sum()

df_movies_combined = pd.concat([df_movies.drop(columns=['genres']), df_movies_one_hot_grouped], axis=1)

df_movies_combined

,movieId,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,toy story,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,jumanji,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,grumpier old men,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,waiting to exhale,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,father of the bride part ii,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,meet the parents,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,3949,requiem for a dream,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3880,3950,tigerland,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3881,3951,two family house,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [5]:
df_ratings = df_ratings.drop(columns=["timestamp"])

df_ratings

,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [6]:
df_users = df_users.drop(columns=["gender", "age", "occupation", "zip"])

In [7]:
df_ratings_sample = df_ratings.sample(n=1_000_000)

df = pd.merge(df_ratings_sample, df_movies_combined, on='movieId')

df

,userId,movieId,rating,title,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,770,437,1,cops and robbersons,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1591,2628,4,star wars: episode i - the phantom menace,1,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,1181,373,3,red rock west,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1077,1617,4,l.a. confidential,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
4,4802,3160,5,magnolia,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,862,1617,3,l.a. confidential,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
999996,4251,2797,2,big,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
999997,204,1396,4,sneakers,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
999998,5047,1917,3,armageddon,1,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [8]:
# We weight the genres by the rating value    
genres = list(df.columns[4:])
for genre in genres:
    df[genre] = df[genre] * df['rating']

df    

,userId,movieId,rating,title,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,770,437,1,cops and robbersons,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1591,2628,4,star wars: episode i - the phantom menace,4,4,0,0,0,0,...,4,0,0,0,0,0,4,0,0,0
2,1181,373,3,red rock west,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
3,1077,1617,4,l.a. confidential,0,0,0,0,0,4,...,0,4,0,0,4,0,0,4,0,0
4,4802,3160,5,magnolia,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,862,1617,3,l.a. confidential,0,0,0,0,0,3,...,0,3,0,0,3,0,0,3,0,0
999996,4251,2797,2,big,0,0,0,0,2,0,...,2,0,0,0,0,0,0,0,0,0
999997,204,1396,4,sneakers,0,0,0,0,0,4,...,0,0,0,0,0,0,4,0,0,0
999998,5047,1917,3,armageddon,3,3,0,0,0,0,...,0,0,0,0,0,0,3,3,0,0


In [9]:
# We normalize the genres
genre_sum = df.groupby('userId')[genres].sum()
genre_count = df.groupby('userId')[genres].apply(lambda x: (x > 0).sum())
genre_norm = genre_sum.div(genre_count, axis=0).fillna(0)

df_features_users = pd.merge(df_ratings_sample, genre_norm, on='userId')

df_features_users

,userId,movieId,rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,770,437,1,2.946721,2.900901,3.268293,2.493976,2.799479,3.389831,3.000000,...,2.642857,2.857143,2.847826,3.090909,3.529412,2.880000,2.831858,3.114504,3.228571,3.600000
1,1591,2628,4,4.285714,4.000000,3.800000,3.700000,4.155039,4.450000,5.000000,...,3.666667,5.000000,3.846154,4.000000,4.333333,4.437500,4.188679,4.521739,4.666667,3.750000
2,1181,373,3,2.802941,2.735849,2.736842,2.417910,2.739872,2.946903,2.809524,...,2.514286,3.354839,2.590361,3.000000,2.833333,2.719577,2.809816,2.900000,3.020408,2.872340
3,1077,1617,4,4.019231,3.785714,2.727273,3.000000,3.617647,4.138889,4.000000,...,2.666667,4.235294,4.240000,3.333333,4.333333,3.892857,4.000000,4.127273,4.142857,4.214286
4,4802,3160,5,3.735043,3.830986,3.705882,3.500000,4.106481,4.016667,5.000000,...,4.333333,4.350000,3.816327,3.913043,4.064516,3.975610,3.793103,4.009615,4.294118,4.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,862,1617,3,3.333333,3.500000,3.666667,2.500000,3.517857,3.750000,4.000000,...,4.000000,3.000000,3.750000,4.000000,3.666667,3.400000,3.666667,3.833333,3.666667,0.000000
999996,4251,2797,2,3.466667,3.285714,3.000000,3.000000,3.465517,4.200000,0.000000,...,3.250000,4.000000,3.000000,0.000000,3.555556,3.500000,3.333333,3.880000,3.666667,0.000000
999997,204,1396,4,3.582822,3.520833,3.380952,3.528302,3.528571,3.606061,3.000000,...,3.782609,3.000000,3.666667,3.800000,3.200000,3.610169,3.468354,3.701149,4.071429,3.571429
999998,5047,1917,3,3.026667,3.114286,2.600000,2.600000,3.201389,3.733333,4.000000,...,3.666667,4.666667,2.800000,3.285714,4.350000,3.702703,3.403846,3.659341,3.909091,3.250000


In [10]:
df_features_movies = pd.merge(df_ratings_sample, df_movies_combined, on='movieId').drop(columns=['rating'])

df_ratings = df_features_users[['userId', 'movieId', 'rating']]

df_features_users = df_features_users.drop(columns=['rating'])

In [11]:
df_features_movies

,userId,movieId,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,770,437,cops and robbersons,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1591,2628,star wars: episode i - the phantom menace,1,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,1181,373,red rock west,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1077,1617,l.a. confidential,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
4,4802,3160,magnolia,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,862,1617,l.a. confidential,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
999996,4251,2797,big,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
999997,204,1396,sneakers,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
999998,5047,1917,armageddon,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [12]:
df_features_users

,userId,movieId,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,770,437,2.946721,2.900901,3.268293,2.493976,2.799479,3.389831,3.000000,3.068027,2.642857,2.857143,2.847826,3.090909,3.529412,2.880000,2.831858,3.114504,3.228571,3.600000
1,1591,2628,4.285714,4.000000,3.800000,3.700000,4.155039,4.450000,5.000000,4.573333,3.666667,5.000000,3.846154,4.000000,4.333333,4.437500,4.188679,4.521739,4.666667,3.750000
2,1181,373,2.802941,2.735849,2.736842,2.417910,2.739872,2.946903,2.809524,2.904362,2.514286,3.354839,2.590361,3.000000,2.833333,2.719577,2.809816,2.900000,3.020408,2.872340
3,1077,1617,4.019231,3.785714,2.727273,3.000000,3.617647,4.138889,4.000000,4.125828,2.666667,4.235294,4.240000,3.333333,4.333333,3.892857,4.000000,4.127273,4.142857,4.214286
4,4802,3160,3.735043,3.830986,3.705882,3.500000,4.106481,4.016667,5.000000,4.198413,4.333333,4.350000,3.816327,3.913043,4.064516,3.975610,3.793103,4.009615,4.294118,4.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,862,1617,3.333333,3.500000,3.666667,2.500000,3.517857,3.750000,4.000000,3.741935,4.000000,3.000000,3.750000,4.000000,3.666667,3.400000,3.666667,3.833333,3.666667,0.000000
999996,4251,2797,3.466667,3.285714,3.000000,3.000000,3.465517,4.200000,0.000000,3.647059,3.250000,4.000000,3.000000,0.000000,3.555556,3.500000,3.333333,3.880000,3.666667,0.000000
999997,204,1396,3.582822,3.520833,3.380952,3.528302,3.528571,3.606061,3.000000,3.676829,3.782609,3.000000,3.666667,3.800000,3.200000,3.610169,3.468354,3.701149,4.071429,3.571429
999998,5047,1917,3.026667,3.114286,2.600000,2.600000,3.201389,3.733333,4.000000,4.094828,3.666667,4.666667,2.800000,3.285714,4.350000,3.702703,3.403846,3.659341,3.909091,3.250000


In [13]:
scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(df_ratings["rating"].to_numpy().reshape(-1, 1))
df_ratings["rating"] = scalerTarget.transform(df_ratings["rating"].to_numpy().reshape(-1, 1))

df_ratings

C:\Users\jujus\AppData\Local\Temp\ipykernel_9552\3202688722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings["rating"] = scalerTarget.transform(df_ratings["rating"].to_numpy().reshape(-1, 1))


,userId,movieId,rating
0,770,437,-1.0
1,1591,2628,0.5
2,1181,373,0.0
3,1077,1617,0.5
4,4802,3160,1.0
...,...,...,...
999995,862,1617,0.0
999996,4251,2797,-0.5
999997,204,1396,0.5
999998,5047,1917,0.0


In [14]:
user_train, user_test = train_test_split(
    df_features_users, train_size=0.80, shuffle=True, random_state=1
)

movie_train, movie_test = train_test_split(
    df_features_movies, train_size=0.80, shuffle=True, random_state=1
)

rating_train, rating_test = train_test_split(
    df_ratings, train_size=0.80, shuffle=True, random_state=1
)

print(user_train)
print(movie_train)
print(rating_train)

        userId  movieId    Action  Adventure  Animation  Children's    Comedy  \
771718    4021     2300  3.573643   3.579268   4.354167    3.841463  3.911504   
521462    5433     2173  2.886364   2.653465   2.363636    2.235294  3.143590   
137361    5535      649  3.634615   3.333333   2.666667    2.294118  3.454545   
404985    4726       48  3.823529   3.866667   4.416667    3.882353  3.769231   
910092    3056      292  3.658065   3.794118   3.944444    4.071429  3.634921   
...        ...      ...       ...        ...        ...         ...       ...   
491263    4064     2970  2.750000   2.760684   3.559322    3.045872  2.974576   
791624     329     3039  2.729032   2.479339   3.310345    2.647887  2.698413   
470924    1057     2288  3.558140   3.457143   3.575000    3.714286  3.870370   
491755    1499     2353  3.756757   3.793103   4.500000    3.727273  4.027778   
128037    2915     2542  3.944444   3.533333   3.500000    3.500000  3.777778   

           Crime  Documenta

The goal of this project is to build a recommendation system.
The recommendation will do the following:
- based on 2 users, recommend a movie that the two users could both like

The 2 users can be either ids or simply made up users from preferences.

Here is the plan:
- We create a movie-movie similarity matrix
- We build a new common-user that will have the average features of the two users
- We get N movies that are the most similar to the movies that the two users have in common
- We estimate the rating of the common-user for each of these N movies (using the neural network)
- We recommend the movie with the highest estimated rating


Building the movie-movie similarity matrix

In [15]:
# Remove the userId column, and remove duplicated movieIds
df_features_movies = df_features_movies.drop(columns=['userId']).drop_duplicates(subset=['movieId'])

# Remove the title column and reset index to movieId
df_features_movies = df_features_movies.drop(columns=['title']).set_index('movieId')

df_features_movies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,
437,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2628,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1617,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
3160,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2619,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3842,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [16]:
# Compute the cosine similarity between the movies
movie_similarity = cosine_similarity(df_features_movies)

movie_similarity = pd.DataFrame(movie_similarity, index=df_features_movies.index, columns=df_features_movies.index)

movie_similarity

movieId,437,2628,373,1617,3160,3240,3004,3328,1356,230,...,3779,3229,651,792,2254,1709,2619,3842,3172,2277
movieId,,,,,,,,,,,,,,,,,,,,,
437,1.0,0.0,0.0,0.0,0.0,1.0,0.707107,0.000000,0.000000,0.000000,...,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2628,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.866025,0.000000,...,0.5,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0
373,0.0,0.0,1.0,0.5,0.0,0.0,0.000000,0.000000,0.000000,0.707107,...,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1617,0.0,0.0,0.5,1.0,0.0,0.0,0.000000,0.353553,0.000000,0.353553,...,0.0,0.353553,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
3160,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.707107,0.000000,0.707107,...,0.0,0.707107,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,0.0,0.0,1.0,0.5,0.0,0.0,0.000000,0.000000,0.000000,0.707107,...,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2619,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.707107,0.000000,0.707107,...,0.0,0.707107,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3842,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
# Function that returns the n most similar movies to a given movie (but not the movie itself)
def get_similar_movies(movieId, n):
    result = movie_similarity.loc[movieId].sort_values(ascending=False).head(n+1)
    try:
        result = result.drop(movieId)
    except:
        pass
    return result.head(n)

get_similar_movies(1, 10)

movieId
2355    1.000000
1064    1.000000
2142    1.000000
3611    1.000000
3754    1.000000
2354    1.000000
3114    1.000000
3751    1.000000
2141    1.000000
588     0.866025
Name: 1, dtype: float64

Getting a common user

In [18]:
# Remove the movieId column, and remove duplicated userIds
df_features_users = df_features_users.drop(columns=['movieId']).drop_duplicates(subset=['userId'])

# Reset index to userId
df_features_users = df_features_users.set_index('userId')

df_features_users

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,
770,2.946721,2.900901,3.268293,2.493976,2.799479,3.389831,3.000000,3.068027,2.642857,2.857143,2.847826,3.090909,3.529412,2.880000,2.831858,3.114504,3.228571,3.600000
1591,4.285714,4.000000,3.800000,3.700000,4.155039,4.450000,5.000000,4.573333,3.666667,5.000000,3.846154,4.000000,4.333333,4.437500,4.188679,4.521739,4.666667,3.750000
1181,2.802941,2.735849,2.736842,2.417910,2.739872,2.946903,2.809524,2.904362,2.514286,3.354839,2.590361,3.000000,2.833333,2.719577,2.809816,2.900000,3.020408,2.872340
1077,4.019231,3.785714,2.727273,3.000000,3.617647,4.138889,4.000000,4.125828,2.666667,4.235294,4.240000,3.333333,4.333333,3.892857,4.000000,4.127273,4.142857,4.214286
4802,3.735043,3.830986,3.705882,3.500000,4.106481,4.016667,5.000000,4.198413,4.333333,4.350000,3.816327,3.913043,4.064516,3.975610,3.793103,4.009615,4.294118,4.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,4.000000,4.000000,4.533333,4.750000,4.857143,0.000000,0.000000,4.500000,0.000000,5.000000,5.000000,4.000000,0.000000,0.000000,3.000000,4.666667,0.000000,0.000000
4525,4.000000,4.000000,0.000000,0.000000,3.800000,3.500000,0.000000,4.090909,3.000000,0.000000,0.000000,0.000000,0.000000,4.000000,3.666667,3.000000,4.214286,0.000000
997,4.111111,4.000000,3.500000,3.500000,3.818182,3.000000,0.000000,4.071429,0.000000,0.000000,2.000000,0.000000,0.000000,3.750000,4.200000,4.142857,4.666667,0.000000


In [19]:
# Function that returns the average features of two users
def get_common_user(user_id1, user_id2):
    return (df_features_users.loc[user_id1] + df_features_users.loc[user_id2]) / 2

def get_common_user_from_preferences(user1_preferences, user2_preferences):
    return (user1_preferences + user2_preferences) / 2

get_common_user(1, 2)

Action         3.850000
Adventure      3.868421
Animation      2.055556
Children's     2.125000
Comedy         3.851429
Crime          3.791667
Documentary    0.000000
Drama          4.163653
Fantasy        3.500000
Film-Noir      2.000000
Horror         1.500000
Musical        2.142857
Mystery        1.666667
Romance        3.687500
Sci-Fi         3.960784
Thriller       3.575269
War            4.366667
Western        2.166667
dtype: float64

## The Neural Network model

In [20]:
user_NN = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ]
)

movie_NN = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ]
)

input_user = tf.keras.layers.Input(shape=(18,))
vu = user_NN(input_user)

input_movie = tf.keras.layers.Input(shape=(18,))
vm = movie_NN(input_movie)

dot = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.models.Model(inputs=[input_user, input_movie], outputs=dot)

model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 1)         │     13,313 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 1)         │     13,313 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ sequential[0][0], │
│                     │                   │            │ sequential_1[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 26,626 (104.01 KB)

 Trainable params: 26,626 (104.01 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Train the model
nb_epochs = 5

model.fit(
        [user_train.loc[:, "Action":].to_numpy(), movie_train.loc[:, "Action":].to_numpy()],
        rating_train.loc[:, "rating":].to_numpy(),
        epochs=nb_epochs,
        validation_data=([user_test.loc[:, "Action":].to_numpy(), movie_test.loc[:, "Action":].to_numpy()], rating_test.loc[:, "rating":].to_numpy()),
    )


Epoch 1/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 36s 1ms/step - loss: 0.2682 - val_loss: 0.2579
Epoch 2/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - loss: 0.2585 - val_loss: 0.2570
Epoch 3/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - loss: 0.2569 - val_loss: 0.2575
Epoch 4/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - loss: 0.2574 - val_loss: 0.2568
Epoch 5/5
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step - loss: 0.2568 - val_loss: 0.2572


In [22]:
def predict_rating(user, movie):
    # If the input is a pandas series, we need to convert it to a dataframe with one row
    if isinstance(user, pd.Series):
        user = pd.DataFrame(user).T
    if isinstance(movie, pd.Series):
        movie = pd.DataFrame(movie).T
    
    # The movie input is a pandas series, we need to convert it to a dataframe with one row
    prediction = model.predict([user.loc[:, "Action":].to_numpy().astype(np.float64), movie.loc[:, "Action":].to_numpy().astype(np.float64)], verbose=0)
    prediction = scalerTarget.inverse_transform(prediction)
    return prediction

predict_rating(user_train.loc[1], movie_train.loc[1])

array([[4.505598]], dtype=float32)

## Evaluation of the model

We put side by side the true rating and the predicted rating

In [23]:
# Evaluate the model
y_pred = model.predict([user_test.loc[:, "Action":].to_numpy(), movie_test.loc[:, "Action":].to_numpy()])
y_pu = scalerTarget.inverse_transform(y_pred)
y_true_u = scalerTarget.inverse_transform(rating_test["rating"].to_numpy().reshape(-1, 1))
result = rating_test.copy()
result["rating"] = y_true_u.flatten()
result["prediction"] = y_pu.flatten()

result

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 6s 997us/step


,userId,movieId,rating,prediction
276826,5382,3527,4.0,4.201886
849425,1987,353,3.0,3.391023
504499,3934,1355,2.0,2.867301
601054,3951,2470,4.0,3.504226
980221,1116,574,5.0,4.238135
...,...,...,...,...
555867,4081,608,5.0,4.454309
30004,3970,1333,4.0,3.149508
124730,3626,3827,4.0,3.382864
195783,2689,2571,5.0,4.454379


In [24]:
# Compute the standard deviation
std = np.std(result["rating"] - result["prediction"])
print("Standard deviation of the model: ", std)

Standard deviation of the model:  1.0107720835667742


## Recommendation system

In [25]:
# Function that returns the 100 most similar movies to the movies that two users have rated (50 each)
def get_most_similars(user_id1, user_id2):
    # Get the movies that user1 and user2 have rated the highest
    user1_movies = df_ratings.loc[df_ratings['userId'] == user_id1].sort_values(by='rating', ascending=False).head(5)
    user2_movies = df_ratings.loc[df_ratings['userId'] == user_id2].sort_values(by='rating', ascending=False).head(5)
    
    # Get the 10 most similar movies to each of the movies that user1 and user2 have rated
    similar_movies = []
    for movieId in user1_movies['movieId']:
        similar_movies.extend(get_similar_movies(movieId, 10).index)
    for movieId in user2_movies['movieId']:
        similar_movies.extend(get_similar_movies(movieId, 10).index)
    
    if user_id1 == user_id2:
        similar_movies = list(set(similar_movies))
    else:
        # Remove the movies that user1 and user2 have rated
        similar_movies = list(set(similar_movies) - set(user1_movies['movieId']) - set(user2_movies['movieId']))
    
    return similar_movies

get_most_similars(1, 2)

[1024,
 3588,
 2821,
 1029,
 1032,
 3594,
 783,
 1681,
 2579,
 661,
 1688,
 2842,
 3741,
 2080,
 2081,
 1061,
 3111,
 2344,
 168,
 2092,
 3759,
 2096,
 3517,
 1213,
 3775,
 3447,
 321,
 2370,
 334,
 592,
 1489,
 594,
 2771,
 1363,
 3805,
 3165,
 3304,
 874,
 1131,
 364,
 2421,
 2422,
 631,
 245,
 1404,
 1533]

In [26]:
# Function that returns the title of a movie given its movieId, to make the recommendation more user-friendly

def get_movie_info(movieId):
    return df_movies.loc[df_movies['movieId'] == movieId]

get_movie_info(1)

,movieId,title,genres
0,1,toy story,"[Animation, Children's, Comedy]"


#### Recommendation function for two users with ids

In [27]:
# Recommandation function
def recommand_common_movie_from_user_ids(user_id1, user_id2):
    common_user = get_common_user(user_id1, user_id2)
    most_similars = get_most_similars(user_id1, user_id2)
    
    best_movie = None
    best_rating = -1
    for movieId in most_similars:
        rating = predict_rating(common_user, df_features_movies.loc[movieId])
        if rating > best_rating:
            best_rating = rating
            best_movie = movieId
            
    return get_movie_info(best_movie)

display(recommand_common_movie_from_user_ids(65, 756))
display(get_common_user(65, 756).sort_values(ascending=False))

,movieId,title,genres
40,41,richard iii,"[Drama, War]"


Crime          4.622222
Horror         4.500000
Musical        4.500000
Thriller       4.375862
War            4.343750
Drama          4.287879
Action         4.269592
Sci-Fi         4.255263
Adventure      4.032353
Comedy         3.785714
Fantasy        3.500000
Romance        3.281250
Documentary    2.500000
Mystery        2.500000
Animation      2.300000
Children's     2.062500
Film-Noir      2.000000
Western        0.000000
dtype: float64

#### Recommendation function for two made up users from their given preferences

In [28]:
# Now, we will create a function to build a user preferences dataframe
def build_user_preferences(
        Action = 0.0,
        Adventure = 0.0,
        Animation = 0.0,
        Childrens = 0.0,
        Comedy = 0.0,
        Crime = 0.0,
        Documentary = 0.0,
        Drama = 0.0,
        Fantasy = 0.0,
        FilmNoir = 0.0,
        Horror = 0.0,
        Musical = 0.0,
        Mystery = 0.0,
        Romance = 0.0,
        SciFi = 0.0,
        Thriller = 0.0,
        War = 0.0,
        Western = 0.0
):
    user_prefs =  pd.DataFrame({
        'Action': [Action],
        'Adventure': [Adventure],
        'Animation': [Animation],
        'Childrens': [Childrens],
        'Comedy': [Comedy],
        'Crime': [Crime],
        'Documentary': [Documentary],
        'Drama': [Drama],
        'Fantasy': [Fantasy],
        'FilmNoir': [FilmNoir],
        'Horror': [Horror],
        'Musical': [Musical],
        'Mystery': [Mystery],
        'Romance': [Romance],
        'SciFi': [SciFi],
        'Thriller': [Thriller],
        'War': [War],
        'Western': [Western]
    })
    
    user_prefs = (user_prefs - 2.5) / 2.5
    
    return user_prefs

user_1 = build_user_preferences(Action=5, Adventure=5, SciFi=5)
user_2 = build_user_preferences(Action=5, Adventure=5, SciFi=5)

In [29]:
# We need to create a function that gives the closest user to a given user preferences
# We will use the cosine similarity for that

def get_closest_user(user_preferences):
    closest_user = None
    closest_similarity = -1
    for userId in df_features_users.index:
        similarity = cosine_similarity(user_preferences, df_features_users.loc[userId].to_numpy().reshape(1, -1))
        if similarity > closest_similarity:
            closest_similarity = similarity
            closest_user = userId
            
    return closest_user

get_closest_user(user_1)

4991

In [30]:
# Now we can create the recommendation function
def recommand_common_movie_from_user_preferences(user1_preferences, user2_preferences):
    closest_user_1 = get_closest_user(user1_preferences)
    closest_user_2 = get_closest_user(user2_preferences)
    
    common_user = get_common_user_from_preferences(user1_preferences, user2_preferences)
    most_similars = get_most_similars(closest_user_1, closest_user_2)
    
    best_movie = None
    best_rating = -1
    for movieId in most_similars:
        rating = predict_rating(common_user, df_features_movies.loc[movieId])
        if rating > best_rating:
            best_rating = rating
            best_movie = movieId
            
    return get_movie_info(best_movie)

recommand_common_movie_from_user_preferences(user_1, user_2)

,movieId,title,genres
3525,3594,center stage,[Drama]


## Testing the recommendation system with random users
We can now test the recommendation system with random users
To see if the recommendation is relevant, we will display the preferences of the two users and the recommendation
The recommendation should be a movie that is similar to the preferences of the two users
If the 2 users prefer comedies, the recommendation should be a comedy, etc.

In [31]:
# We will create two random users and recommend a movie for them
for i in range(10):
    user_1 = build_user_preferences(
        Action=np.random.randint(0, 6),
        Adventure=np.random.randint(0, 6),
        Animation=np.random.randint(0, 6),
        Childrens=np.random.randint(0, 6),
        Comedy=np.random.randint(0, 6),
        Crime=np.random.randint(0, 6),
        Documentary=np.random.randint(0, 6),
        Drama=np.random.randint(0, 6),
        Fantasy=np.random.randint(0, 6),
        FilmNoir=np.random.randint(0, 6),
        Horror=np.random.randint(0, 6),
        Musical=np.random.randint(0, 6),
        Mystery=np.random.randint(0, 6),
        Romance=np.random.randint(0, 6),
        SciFi=np.random.randint(0, 6),
        Thriller=np.random.randint(0, 6),
        War=np.random.randint(0, 6),
        Western=np.random.randint(0, 6)
    )
    user_2 = build_user_preferences(
        Action=np.random.randint(0, 6),
        Adventure=np.random.randint(0, 6),
        Animation=np.random.randint(0, 6),
        Childrens=np.random.randint(0, 6),
        Comedy=np.random.randint(0, 6),
        Crime=np.random.randint(0, 6),
        Documentary=np.random.randint(0, 6),
        Drama=np.random.randint(0, 6),
        Fantasy=np.random.randint(0, 6),
        FilmNoir=np.random.randint(0, 6),
        Horror=np.random.randint(0, 6),
        Musical=np.random.randint(0, 6),
        Mystery=np.random.randint(0, 6),
        Romance=np.random.randint(0, 6),
        SciFi=np.random.randint(0, 6),
        Thriller=np.random.randint(0, 6),
        War=np.random.randint(0, 6),
        Western=np.random.randint(0, 6)
    )
    reco = recommand_common_movie_from_user_preferences(user_1, user_2)
    print("User 1: ")
    display(user_1)
    print("User 2: ")
    display(user_2)
    print("Recommendation: ")
    display(reco)
    print(" ----- ")

User 1: 


,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,0.2,1.0,0.2,0.2,-0.6,-0.2,0.6,-0.6,0.6,0.6,-0.6,-0.2,0.2,0.2,-1.0,0.6,-0.6,-1.0


User 2: 


,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,-1.0,1.0,-1.0,0.6,-0.2,1.0,1.0,-0.2,0.2,0.6,0.6,-0.2,-0.6,1.0,0.6,0.6,-0.6,-0.6


Recommendation: 


...

User 2: 


,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,-0.6,1.0,-1.0,-0.6,0.6,1.0,0.2,0.6,1.0,-0.6,-1.0,0.6,-0.2,-1.0,-0.2,0.2,0.2,-0.6


Recommendation: 


,movieId,title,genres
3284,3353,the closer you get,"[Comedy, Romance]"


 ----- 


## Conclusion

We can see that the recommendation system works most of the time.
But there are some cases where the recommendation is not relevant.

We can also criticize the model itself not being that good, as the standard deviation of the model is quite high.
The thing is, even with higher epochs, the model doesn't seem to improve that much, and the loss doesn't decrease that much.

We could try to improve the model by adding more layers or more neurons, but as the dataset is quite small, and the training time is already quite long, we will not have time to improve this model, unfortunately.